In [10]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [11]:
import IPython.display as ipd
import numpy as np, pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers.file_utils import ModelOutput
from transformers import AutoConfig, Wav2Vec2Processor
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2PreTrainedModel,
    Wav2Vec2Model
)

from pathlib import Path
from tqdm import tqdm
import torchaudio, os, sys, json, pickle, librosa

from dataclasses import dataclass
from typing import Optional, Tuple
from tqdm import tqdm

In [12]:


@dataclass
class SpeechClassifierOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None
    hidden_rep: Optional[Tuple[torch.FloatTensor]] = None



class Wav2Vec2ClassificationHead(nn.Module):
    """Head for wav2vec classification task."""

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(config.final_dropout)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x0 = self.dropout(x)
#         print('----------------------------')
#         print(x0[:,-10:])
#         print(x0.shape)
#         print('----------------------------')
        x1 = self.out_proj(x0)
        return x0, x1

class Wav2Vec2ForSpeechClassification(Wav2Vec2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.pooling_mode = config.pooling_mode
        self.config = config

        self.wav2vec2 = Wav2Vec2Model(config)
        self.classifier = Wav2Vec2ClassificationHead(config)

        self.init_weights()

    def freeze_feature_extractor(self):
        self.wav2vec2.feature_extractor._freeze_parameters()

    def merged_strategy(self, hidden_states, mode="mean"):
        if mode == "mean":
            outputs = torch.mean(hidden_states, dim=1)
        elif mode == "sum":
            outputs = torch.sum(hidden_states, dim=1)
        elif mode == "max":
            outputs = torch.max(hidden_states, dim=1)[0]
        else:
            raise Exception("The pooling method hasn't been defined! Your pooling mode must be one of these ['mean', 'sum', 'max']")

        return outputs

    def forward(
            self,
            input_values,
            attention_mask=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
            labels=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        outputs = self.wav2vec2(
            input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        hidden_states = outputs[0]
        hidden_states = self.merged_strategy(hidden_states, mode=self.pooling_mode)
        hidden_rep, logits = self.classifier(hidden_states)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)

        if not return_dict:
            output = (hidden_rep + logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SpeechClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
            hidden_rep=hidden_rep
        )

In [13]:
!export CUDA_VISIBLE_DEVICES=1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.current_device()

0

In [14]:
# model_name_or_path = "/mnt/data/aman/mayank/MTP/mount_points/jan_19/Error-Driven-ASR-Personalization/MCV_accent/data/dristi_accent-recognition/checkpoint-6400/"
model_name_or_path = "/home/mayank/MTP/begin_again/Error-Driven-ASR-Personalization/MCV_accent/data/dristi_accent-recognition/checkpoint-6400/"
config = AutoConfig.from_pretrained(model_name_or_path)
processor = Wav2Vec2Processor.from_pretrained(model_name_or_path)
sampling_rate = processor.feature_extractor.sampling_rate
model = Wav2Vec2ForSpeechClassification.from_pretrained(model_name_or_path).to(device)

In [15]:
def speech_file_to_array_fn(path, sampling_rate):
    speech_array, _sampling_rate = torchaudio.load(path)
    resampler = torchaudio.transforms.Resample(_sampling_rate)
    speech = resampler(speech_array).squeeze().numpy()
    return speech

def predict(path, sampling_rate):
#     print(path)
    speech = speech_file_to_array_fn(path, sampling_rate)
    features = processor(speech, sampling_rate=sampling_rate, return_tensors="pt", padding=True)
    input_values = features.input_values.to(device)
    attention_mask = None

    with torch.no_grad():
        op = model(input_values, attention_mask=attention_mask)
        logits = op.logits
        hidden_rep = op.hidden_rep
        
    scores = F.softmax(logits, dim=1).detach().cpu().numpy()[0]
    outputs = [{"Accent": config.id2label[i], "Score": f"{round(score * 100, 3):.1f}%"} for i, score in enumerate(scores)]
    return outputs, hidden_rep

def prediction(df_row):
    if 'path' in df_row: path = df_row["path"]
    else: path = df_row["audio_filepath"]
    speech, sr = torchaudio.load(path)
    speech = speech[0].numpy().squeeze()
    speech = librosa.resample(np.asarray(speech), sr, sampling_rate)
    outputs, hidden_rep = predict(path, sampling_rate)
#     print(hidden_rep[:,-10:])
    return hidden_rep

def extract_features(file_list, file_dir):
    with open(file_dir.replace('.json', '_w2v2.file'), 'wb') as f:
        for file in file_list:
            w2v2_features = prediction(file).cpu().detach().numpy()
            pickle.dump(w2v2_features, f)

In [ ]:
jsons_path = '.json'
jsons = [f.name for f in os.scandir('../MCV_accent/invalidated/') if '.json' in f.name and f.name.split('.')[0] not in ['unlabelled',
                                                                                                              'other']]
print(jsons)

for file in tqdm(jsons):
    print('_'*20)
    
    
    json_file_path = '../MCV_accent/invalidated/' + file 
    json_file = open(json_file_path)
    json_list = [json.loads(line.strip()) for line in json_file]
    print(json_file_path)
    
    extract_features(json_list, json_file_path)
    print(len(json_list))
    print('_'*20, '\n\n')

['philippines.json', 'indian.json', 'australia.json', 'us.json', 'england.json', 'canada.json', 'african.json']


  0%|          | 0/7 [00:00<?, ?it/s]

____________________
../MCV_accent/invalidated/philippines.json


 14%|█▍        | 1/7 [07:12<43:15, 432.56s/it]

1500
____________________ 


____________________
../MCV_accent/invalidated/indian.json


In [ ]:
from pydub import AudioSegment

for fxn in ['FL2MI', 'GCMI', 'LogDMI']:
    for run in [1, 2, 3]:
        json_file_path = f'../mz-expts/african/manifests/TSS_output/all/budget_100/target_10/{fxn}/eta_1.0/euclidean/w2v2/run_{run}/train/train.json'
        json_file = open(json_file_path)
        json_list = [json.loads(line.strip()) for line in json_file]
        count = 0
        alarm = 0
        for f in json_list:
            if not os.path.exists(f['audio_filepath']):
                mp3path = f['audio_filepath']
                mp3path = path.replace('/wav/', '/clips/')
                mp3path = path.replace('wav', 'mp3')
                count += 1
                if not os.path.exists(mp3path):
                    alarm += 1
                sound = AudioSegment.from_mp3(mp3path)
                sound.export(f['audio_filepath'], format="wav")
        print(len(json_list))
        print(count)
        print(alarm)

In [ ]:
for f in json_list:
    if not os.path.exists(f['audio_filepath']):
        count += 1
print(len(json_list))
print(count)